# I'm going to do this in Pytorch 

TF is refusing to recongise cuda, but pytorch is.

Going to have to use it so that this model can be trained in a resonable time. 

There is some issue with the configuration on my computer, and these models are too big to be worked on in colab without colab timing out. 

Pytorch is the standard when it comes to research, so I use it unapologetically.

In [1]:
# Beautiful Import errors from TF
import torch
import tensorflow as tf

/home/jarvis/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jarvis/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jarvis/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jarvis/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
tf.test.is_gpu_available()

False

In [3]:
torch.cuda.is_available()

True

## Imports

Because every project starts here :)

In [4]:
import numpy as np
import torch
import torchvision
from torch import nn as nn
from torch.nn import functional as F
from torchvision import models
from torch import optim
from torchsummary import summary
import time

In [5]:
# defining our network

class VGG(nn.Module):
  def __init__(self, num_classes):
    super(VGG, self).__init__()

    self.num_classes = num_classes
    
    # our pretrained model
    model = models.vgg19(pretrained = True)
    self.features = model.features

    self.fc1 = nn.Linear(51200, 512)
    self.d1 = nn.Dropout(.2)
    self.fc2 = nn.Linear(512, 64)
    self.d2 = nn.Dropout(.4)
    self.f3 = nn.Linear(64, num_classes)

  def forward(self, x):
    # get features flatten1
    x = self.features(x)
    x = x.view(x.size(0), -1)
    
    x = F.relu(self.fc1(x))
    x = self.d1(x)
    x = F.relu(self.fc2(x))
    x = self.d2(x)
    return F.softmax(self.f3(x))

device = "cuda:0"

model = VGG(6)
model = model.to(device)

In [6]:
#this is to get our data

def load_train():
    data_path = 'train'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    
    # to prove that I got rid of the civilian cars
    print(train_dataset.classes)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=8,
        num_workers=0,
        shuffle=True
    )
    return train_loader

def load_val():
    data_path = 'test'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    
    # to prove that I got rid of the civilian cars
    print(train_dataset.classes)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=8,
        num_workers=0,
        shuffle=True
    )
    return train_loader

train = load_train()
val = load_val()

['BMP2', 'Background', 'Buk-M1-2', 'T14', 'T90', 'ZSU23']
['BMP2', 'Background', 'Buk-M1-2', 'T14', 'T90', 'ZSU23']


In [7]:
# we're going to save the best model for later
best_acc = 0.0

# history so we can GRAPH
history = []

# number of epochs
num_epochs = 30

# opti and crit
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

print('| Epoch\t | Train Loss\t| Train Acc\t| Valid Loss\t| Valid Acc\t| Time')

start = time.time()
for epoch in range(num_epochs): 
    
    epoch_start = time.time()
    running_corrects = 0.0
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # statistics
        preds = torch.round(outputs)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels[0])
        
        # status bar
        if i % 70 == 0:    
            print("-", end="")
            
    train_loss = running_loss / (len(train.dataset))
    train_acc = running_corrects / (len(train.dataset))
    
    running_corrects = 0.0
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(val):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)

        # statistics
        preds = torch.round(outputs)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels[0])
        
    val_loss = (running_loss )/ (len(val.dataset))
    val_acc = running_corrects / (len(val.dataset))
    
    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start
    
    # save model if best
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = model.state_dict()
    
    # print out results so we can see in real time
    print("")
    print("| {}\t | {:.4f}\t| {:.4f}\t| {:.4f}\t| {:.4f}\t| {:.4f}".format(epoch + 1, train_loss, train_acc, val_loss, val_acc, epoch_time))
    
    # save our history
    tmp = [train_loss, train_acc, val_loss, val_acc]
    history.append(tmp)
    
end = time.time()
print("")
print(f'Finished Training after {end-start}')

| Epoch	 | Train Loss	| Train Acc	| Valid Loss	| Valid Acc	| Time


/usr/lib/python3/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


---------------------------------------------------------------------------
| 1	 | 1.8747	| 0.9894	| 2.0436	| 0.9778	| 1752.5857
---------------------------------------------------------------------------
| 2	 | 1.8749	| 0.9528	| 2.0436	| 0.0889	| 1759.3044
---------------------------------------------------------------------------
| 3	 | 1.8765	| 0.9821	| 2.0436	| 0.9111	| 1765.4253
---------------------------------------------------------------------------
| 4	 | 1.8767	| 1.0154	| 1.9186	| 0.3556	| 1767.5522
---------------------------------------------------------------------------
| 5	 | 1.8741	| 1.0200	| 1.9186	| 1.0667	| 1769.1581
---------------------------------------------------------------------------
| 6	 | 1.8741	| 0.9804	| 1.7936	| 0.2667	| 1764.5126
---------------------------------------------------------------------------
| 7	 | 1.8756	| 0.9796	| 1.9186	| 1.1556	| 1766.5298
---------------------------------------------------------------------------
| 8	 | 1.8739	| 1.003

In [8]:
# Save the best model
torch.save(best_model_wts, "./BestVGGModel")

# Trying a standard CNN 

I will not be using VGG as a pre-trained model for feature extraction, will use bog-standard CNN with hopefully the same results!


In [9]:
import numpy as np
import torch
import torchvision
from torch import nn as nn
from torch.nn import functional as F
from torchvision import models
from torch import optim
from torchsummary import summary
import time

In [10]:
#this is to get our data

def load_train():
    data_path = 'train'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    
    # to prove that I got rid of the civilian cars
    print(train_dataset.classes)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    return train_loader

def load_val():
    data_path = 'test'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    
    # to prove that I got rid of the civilian cars
    print(train_dataset.classes)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    return train_loader

train = load_train()
val = load_val()

['BMP2', 'Background', 'Buk-M1-2', 'T14', 'T90', 'ZSU23']
['BMP2', 'Background', 'Buk-M1-2', 'T14', 'T90', 'ZSU23']


In [11]:
class BoganNetwork(nn.Module):
  def __init__(self, num_classes):
    super(BoganNetwork, self).__init__()

    self.num_classes = num_classes

    self.conv1 = nn.Conv2d(3, 64, 3)
    self.d1 = nn.Dropout(.2)
    self.pool = nn.AdaptiveAvgPool2d(5)
    self.conv2 = nn.Conv2d(64, 32, 2)
    self.d2 = nn.Dropout(.3)
    self.fc1 = nn.Linear(512, 512)
    self.d3 = nn.Dropout(.5)
    self.fc2 = nn.Linear(512, num_classes)

  def forward(self, x):
    # get features flatten1
    x = F.relu(self.conv1(x))
    x = self.d1(x)
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.d2(x)
    x = x.view(x.size(0), -1)
    x = F.relu(self.fc1(x))
    x = self.d3(x)
    x = self.fc2(x)
    return F.softmax(x)

In [12]:
device = "cuda"

model_new = BoganNetwork(6)
model_new = model_new.to(device)

In [13]:
# we're going to save the best model for later
best_acc = 0.0

# history so we can GRAPH
history = []

# number of epochs
num_epochs = 30

# opti and crit
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_new.parameters(), lr = 0.0005)

print('| Epoch\t | Train Loss\t| Train Acc\t| Valid Loss\t| Valid Acc\t| Time')

start = time.time()
for epoch in range(num_epochs): 
    
    epoch_start = time.time()
    running_corrects = 0.0
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_new(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # statistics
        preds = torch.round(outputs)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels[0])
        
        # status bar
        if i % 40 == 0:    
            print("-", end="")
            
    train_loss = running_loss / (len(train.dataset))
    train_acc = running_corrects / (len(train.dataset))
    
    running_corrects = 0.0
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(val):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model_new(inputs)

        # statistics
        preds = torch.round(outputs)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels[0])
        
    val_loss = (running_loss )/ (len(val.dataset))
    val_acc = running_corrects / (len(val.dataset))
    
    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start
    
    # save model if best
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = model_new.state_dict()
    
    # print out results so we can see in real time
    print("")
    print("| {}\t | {:.4f}\t| {:.4f}\t| {:.4f}\t| {:.4f}\t| {:.4f}".format(epoch + 1, train_loss, train_acc, val_loss, val_acc, epoch_time))
    
    # save our history
    tmp = [train_loss, train_acc, val_loss, val_acc]
    history.append(tmp)
    
end = time.time()
print("")
print(f'Finished Training after {end-start}')

| Epoch	 | Train Loss	| Train Acc	| Valid Loss	| Valid Acc	| Time
-

/usr/lib/python3/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


-----------------------------------------------------------------
| 1	 | 1.5140	| 1.0382	| 1.4645	| 0.4667	| 219.0180
------------------------------------------------------------------
| 2	 | 1.3859	| 0.9551	| 1.2223	| 1.4444	| 219.0096
------------------------------------------------------------------
| 3	 | 1.3213	| 0.9854	| 1.2796	| 0.1778	| 218.9087
------------------------------------------------------------------
| 4	 | 1.2734	| 1.0172	| 1.1341	| 1.2333	| 218.9198
------------------------------------------------------------------
| 5	 | 1.2355	| 0.9896	| 1.2891	| 0.7222	| 218.8991
------------------------------------------------------------------
| 6	 | 1.2085	| 0.9892	| 1.1408	| 1.4556	| 218.6547
------------------------------------------------------------------
| 7	 | 1.1896	| 0.9774	| 1.1653	| 0.1667	| 218.6734
------------------------------------------------------------------
| 8	 | 1.1730	| 1.0400	| 1.0450	| 1.0556	| 218.4293
-------------------------------------------------

In [14]:
# Save the best model
torch.save(best_model_wts, "./BestCNNModel")

# Conculsion

Seems like the standard CNN model did better when comparing the best model it produced to the best model that the best VGG model, however I think that this can be a bit misleading. 

The Validation set was very small, and had data that was not as mixed (having both the images from the game and real life), so it was hard to see if a model truly generalised or simply just got lucky. With a larger training set, it should be more clear if one model is truly better than another in the sense of accuracy. Another change would be to have two validation sets, one with the synthetic data and one with the real-life data. This could tell us if the model was generalising in terms of the synthetic and/or real data, which would be useful to know for gauging the overeall performance of the model.

Another issue was training time, and GPU memory limitations. I trained on a GTX 980ti graphics card with 6 gb of memory. I was having issues with my card not having enough memory to hold both the VGG model and a batch larger then 8. Those smaller batch sizes could have affected learning, making it not a fair apples to apples comparison to the standard CNN model. Being on a older card, The VGG model took a very long time to train in comparison to the standard CNN model. Luckly for me, my commputer turned out to be a very nice space heater for the recent cold front, keep me very warm these last few days :) 